# Apache Parquet vs CSV - Benchmarking and Visualization

This notebook demonstrates the advantages of using Apache Parquet over traditional CSV files for tabular data storage and analytics. Using a real-world books dataset, we compare file sizes and query performance between CSV and Parquet formats (including compressed and partitioned variants). Visualizations and benchmarks illustrate how Parquet can significantly reduce storage requirements and speed up data processing, especially for columnar queries and filtered reads.

### 1. Setup and Data Preparation
Load necessary libraries, setup paths and load dataset.

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Path setup
csv_file = "data/books.csv"
parquet_default = "data/books_default.parquet"
parquet_compressed = "data/books_compressed.parquet"
parquet_partitioned = "data/books_partitioned"

# Load CSV
print("Loading CSV...")
df = pd.read_csv(csv_file)
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")

### 2. Export Dataset to Different Parquet Formats
- Parquet with default settings.
- Parquet with ZStandard compression.
- Parquet with ZStandard compression and partitioning.

In [ ]:
print("Exporting books CSV dataset to Parquet files...")

df.to_parquet(parquet_default, engine="fastparquet", index=False)
print(f"Parquet file created with default settings: {parquet_default}")

df.to_parquet(parquet_compressed, engine="fastparquet", compression="zstd", index=False)
print(f"Parquet file created with ZStandard compression: {parquet_compressed}")

df.to_parquet(parquet_partitioned, engine="fastparquet", compression="zstd", partition_cols=["language"], index=False)
print(f"Parquet files created with ZStandard compression and partitioning by language: {parquet_partitioned}")

### 3. Compare File Sizes
Calculate and visualize file sizes for CSV and different Parquet formats.

In [ ]:
# Compare File Sizes
def file_size(path):
    if os.path.isdir(path):
        return sum(os.path.getsize(os.path.join(root, f))
                   for root, _, files in os.walk(path) for f in files)
    else:
        return os.path.getsize(path)

sizes = {
    "CSV": file_size(csv_file),
    "Parquet (default)": file_size(parquet_default),
    "Parquet (compressed)": file_size(parquet_compressed),
    "Parquet (comp/partioned)": file_size(parquet_partitioned),
}

sizes_mb = {k: v/1024/1024 for k, v in sizes.items()}
csv_size = sizes_mb["CSV"]

print(" File Size Comparison ".center(51, '='))

for name, size in sizes_mb.items():
    if name == "CSV":
        print(f"{name:<25} {size:.2f} MB")
    else:
        reduction = (1 - size / csv_size) * 100
        print(f"{name:<25} {size:.2f} MB  ({reduction:.1f}% smaller)")

# Plot file size comparison
plt.figure(figsize=(10,6))
plt.bar(sizes_mb.keys(), sizes_mb.values(), color=["#f39c12","#2980b9","#8e44ad","#27ae60"])
plt.ylabel("File Size (MB)")
plt.title("File Size Comparison - CSV vs Parquet")
plt.tight_layout()
plt.show()

### 4. Compare Performance
Measure and visualize read times for CSV and different Parquet formats, including filtered queries.

In [ ]:
# Helper function for benchmarking
def benchmark(description, func):
    """Runs a benchmark measuring execution time and memory usage."""
    start = time.time()
    _ = func()
    end = time.time()
    elapsed = end - start
    print(f"{description:<45} "
          f"Time: {elapsed:.3f}s")
    return elapsed

def print_percentage_faster(csv_benchmark, parquet_benchmark):
    reduction = (1 - parquet_benchmark / csv_benchmark) * 100
    print(f" Parquet is {reduction:.1f}% faster than CSV".rjust(58, "="))

benchmarks = {}

#### 4.1 First Case: Load Full Dataset
Benchmark read times when reading the entire dataset.

In [ ]:
print(" Load Full Dataset ".center(58, "="))

benchmarks["CSV - Full Load"] = benchmark(
    "CSV", 
    lambda: pd.read_csv(csv_file))

benchmarks["Parquet - Full Load"] = benchmark(
    "Parquet", 
    lambda: pd.read_parquet(parquet_default, engine="fastparquet"))

print_percentage_faster(benchmarks["CSV - Full Load"], benchmarks["Parquet - Full Load"])

#### 4.2 Second Case: Column pruning (title, author, rating)
Benchmark read times when reading only specific columns.

In [ ]:
print("\n" + " Load Subset of Columns (title, author, rating) ".center(58, "="))

benchmarks["CSV - Subset Columns"] = benchmark(
    "CSV - Column Subset", 
    lambda: pd.read_csv(csv_file, usecols=["title", "author", "rating"]))

benchmarks["Parquet - Column Pruning"] = benchmark(
    "Parquet - Column Pruning", 
    lambda: pd.read_parquet(parquet_default, columns=["title", "author", "rating"], engine="fastparquet"))

print_percentage_faster(benchmarks["CSV - Subset Columns"], benchmarks["Parquet - Column Pruning"])

#### 4.3 Third Case: Predicate pushdown (rating > 4.5)
Benchmark read times when applying a filter on the dataset after loading (CSV) vs predicate pushdown (Parquet).

In [ ]:
print("\n" + " Load Rows Where: rating > 4.5 ".center(58, "="))

benchmarks["CSV - Filter Rows"] = benchmark(
    "CSV - Filter Rows After Load", 
    lambda: pd.read_csv(csv_file)[lambda d: d["rating"] > 4.5])

benchmarks["Parquet - Predicate Pushdown"] = benchmark(
    "Parquet - Predicate Pushdown", 
    lambda: pd.read_parquet(parquet_default, filters=[("rating", ">", 4.5)], engine="fastparquet"))

print_percentage_faster(benchmarks["CSV - Filter Rows"], benchmarks["Parquet - Predicate Pushdown"])

#### 4.4 Fourth Case: Partition pruning (English books only)
Benchmark read times when applying a filter on the dataset after loading (CSV) vs loading a partitioned column (Parquet).

In [ ]:
print("\n" + " Load Rows Where: language == 'English' ".center(58, "="))

benchmarks["CSV - Filter by Language"] = benchmark(
    "CSV - Filter Rows After Load", 
    lambda: pd.read_csv(csv_file)[lambda d: d["language"] == "English"])

benchmarks["Parquet - Read Language Partition"] = benchmark(
    "Parquet - Partition Pruning", 
    lambda: pd.read_parquet(os.path.join(parquet_partitioned, "language=English"), engine="fastparquet"))

print_percentage_faster(benchmarks["CSV - Filter by Language"], benchmarks["Parquet - Read Language Partition"])

### 5. Benchmark Summary
Summarize and visualize all benchmark results for easy comparison.

In [ ]:
# Group benchmarks by case
cases = [
    ("Full Load", "CSV - Full Load", "Parquet - Full Load"),
    ("Column Pruning", "CSV - Subset Columns", "Parquet - Column Pruning"),
    ("Predicate Pushdown", "CSV - Filter Rows", "Parquet - Predicate Pushdown"),
    ("Partition Pruning", "CSV - Filter by Language", "Parquet - Read Language Partition"),
]

csv_times = [benchmarks[case[1]] for case in cases]
parquet_times = [benchmarks[case[2]] for case in cases]
labels = [case[0] for case in cases]
x = np.arange(len(labels))
width = 0.35

plt.figure(figsize=(10,6))
plt.bar(x - (width / 2), csv_times, width, label='CSV', color="#f39c12")
plt.bar(x + (width / 2), parquet_times, width, label='Parquet', color="#27ae60")
plt.ylabel("Time (seconds)")
plt.title("Performance Comparison - CSV vs Parquet")
plt.xticks(x, labels)
plt.legend(loc='upper right', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()